In [1]:
import os
from tqdm import tqdm
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn

from hparams import EMOTIONX_MODEL_HPARAMS
from pytorch_pretrained_bert import BertTokenizer
from models import EmotionX_Model
from utils import load_data, shuffle_trainset, get_batch, make_dir, print_params

import csv
import json

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.manual_seed(0)

In [3]:
def train_start():
    if not torch.cuda.is_available():
        raise NotImplementedError()
    hparams = type('', (object,), EMOTIONX_MODEL_HPARAMS)() # dict to class

    # data
    fr_train_dialogs, fr_train_labels = load_data(hparams, hparams.fr_train_path)
    test_dialogs, test_labels = load_data(hparams, hparams.fr_test_path)
    train_dialogs = fr_train_dialogs
    train_labels = fr_train_labels
    assert len(train_dialogs) == len(train_labels)
    assert len(test_dialogs) == len(test_labels)

    # hyper-parameter
    hparams.n_appear = [sum(train_labels, []).count(i) for i in range(8)]
    max_i = len(train_dialogs)
    total_step = 0
    print_per = len(train_dialogs)
    highest_micro_f1 = 0.

    # model
    model = EmotionX_Model(hparams)
    model.cuda()
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), hparams.learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_i)
    writer = SummaryWriter(log_dir=hparams.log_dir)

    # train
    for i_epoch in range(hparams.n_epoch):
        train_dialogs, train_labels = shuffle_trainset(train_dialogs, train_labels)
        scheduler.step()

        for i_step in tqdm(range(max_i)):
            batch_dialogs = get_batch(train_dialogs, hparams.batch_size, i_step)
            batch_labels = get_batch(train_labels, hparams.batch_size, i_step)
            optimizer.zero_grad()
            pred_labels = model(batch_dialogs)
            loss = model.cal_loss(batch_labels, pred_labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), hparams.clip)
            optimizer.step()

            # print
            if i_step % print_per == 0:
                model.eval()
                n_appear = [0] * (hparams.n_class - 1)
                n_correct = [0] * (hparams.n_class - 1)
                n_positive = [0] * (hparams.n_class - 1)
                for i_test in range(len(test_dialogs) // hparams.batch_size):
                    batch_dialogs = get_batch(test_dialogs, hparams.batch_size, i_test)
                    batch_labels = get_batch(test_labels, hparams.batch_size, i_test)
                    pred_labels = model(batch_dialogs)
                    counts = model.count_for_eval(batch_labels, pred_labels)
                    n_appear = [x + y for x, y in zip(n_appear, counts[0])]
                    n_correct = [x + y for x, y in zip(n_correct, counts[1])]
                    n_positive = [x + y for x, y in zip(n_positive, counts[2])]
                uwa, wa = model.get_uwa_and_wa(n_appear, n_correct)
                precision, recall, f1, micro_f1, macro_f1 = model.get_f1_scores(
                    n_appear, n_correct, n_positive)

                print('i_epoch: ', i_epoch)
                print('i_total_step: ', total_step)
                print('n_true:\t\t\t', n_appear)
                print('n_positive:\t\t', n_positive)
                print('n_true_positive:\t', n_correct)
                print('precision:\t[%.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f]' % (
                    precision[0], precision[1], precision[2], precision[3], 
                    precision[4], precision[5], precision[6]))
                print('recall:\t\t[%.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f]' % (
                    recall[0], recall[1], recall[2], recall[3], 
                    recall[4], recall[5], recall[6]))
                print('f1:\t\t[%.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f]' % (
                    f1[0], f1[1], f1[2], f1[3], f1[4], f1[5], f1[6]))
                if micro_f1 > highest_micro_f1:
                    highest_micro_f1 = micro_f1
                    torch.save(model.state_dict(), "en_model_save/en_model_save_acc_06_26" + str(highest_micro_f1))
                    friend_high_step = total_step
                print('Micro F1: %.4f (<=%.4f at %d-th total_step)'
                    % (micro_f1, highest_micro_f1, friend_high_step))
                print()

                # write
                writer.add_scalar(hparams.log_micro_f1+'fr', micro_f1, total_step)
                writer.add_scalar(hparams.log_wce_loss+'fr', loss, total_step)
                total_step += 1

                model.train()

In [4]:
torch.cuda.empty_cache()

In [5]:
train_start()

[6039, 1587, 436, 217, 674, 1506, 308, 2558]


/home/key13689/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|          | 0/920 [00:00<?, ?it/s]

i_epoch:  0
i_total_step:  0
n_true:			 [1287, 304, 85, 32, 161, 286, 68]
n_positive:		 [929, 0, 1206, 83, 1, 4, 0]
n_true_positive:	 [488, 0, 45, 3, 0, 0, 0]
precision:	[0.5253, 0.0000, 0.0373, 0.0361, 0.0000, 0.0000, 0.0000]
recall:		[0.3792, 0.0000, 0.5294, 0.0938, 0.0000, 0.0000, 0.0000]
f1:		[0.4404, 0.0000, 0.0697, 0.0522, 0.0000, 0.0000, 0.0000]


  0%|          | 2/920 [00:09<1:39:38,  6.51s/it]

Micro F1: 0.2411 (<=0.2411 at 0-th total_step)



...

  0%|          | 0/920 [00:00<?, ?it/s]

i_epoch:  9
i_total_step:  9
n_true:			 [1287, 304, 85, 32, 161, 286, 68]
n_positive:		 [1168, 348, 108, 0, 232, 332, 35]
n_true_positive:	 [1124, 286, 78, 0, 149, 258, 11]
precision:	[0.9623, 0.8218, 0.7222, 0.0000, 0.6422, 0.7771, 0.3143]
recall:		[0.8733, 0.9408, 0.9176, 0.0000, 0.9255, 0.9021, 0.1618]
f1:		[0.9157, 0.8773, 0.8083, 0.0000, 0.7583, 0.8350, 0.2136]


  0%|          | 2/920 [00:09<1:38:08,  6.41s/it]

Micro F1: 0.8574 (<=0.8574 at 9-th total_step)



100%|██████████| 920/920 [03:27<00:00,  4.43it/s]


In [ ]:
def test(path):
    if not torch.cuda.is_available():
        raise NotImplementedError()
    hparams = type('', (object,), EMOTIONX_MODEL_HPARAMS)() # dict to class

    # data
    fr_train_dialogs, fr_train_labels = load_data(hparams, hparams.fr_train_path)
    train_dialogs = fr_train_dialogs 
    train_labels = fr_train_labels
    test_dialogs, test_labels = load_data(hparams, hparams.fr_test_path)
    assert len(test_dialogs) == len(test_labels)

    # hyper-parameter
    hparams.n_appear = [sum(train_labels, []).count(i) for i in range(8)]
    max_i = len(train_dialogs)
    total_step = 0
    print_per = len(train_dialogs)
    highest_micro_f1 = 0.
    hparams.dropout = 0

    # model
    model = EmotionX_Model(hparams)
    model.cuda()
    model.load_state_dict(torch.load(path))
    model.eval()
    optimizer = torch.optim.Adam(model.parameters(), hparams.learning_rate)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_i)
    writer = SummaryWriter(log_dir=hparams.log_dir)

    # test
    n_appear = [0] * (hparams.n_class - 1)
    n_correct = [0] * (hparams.n_class - 1)
    n_positive = [0] * (hparams.n_class - 1)
    for i_test in range(len(test_dialogs) // hparams.batch_size):
        batch_dialogs = get_batch(test_dialogs, hparams.batch_size, i_test)
        batch_labels = get_batch(test_labels, hparams.batch_size, i_test)
        pred_labels = model(batch_dialogs)
        counts = model.count_for_eval(batch_labels, pred_labels)
        n_appear = [x + y for x, y in zip(n_appear, counts[0])]
        n_correct = [x + y for x, y in zip(n_correct, counts[1])]
        n_positive = [x + y for x, y in zip(n_positive, counts[2])]
        uwa, wa = model.get_uwa_and_wa(n_appear, n_correct)
        precision, recall, f1, micro_f1, macro_f1 = model.get_f1_scores(
            n_appear, n_correct, n_positive)

    print('n_true:\t\t\t', n_appear)
    print('n_positive:\t\t', n_positive)
    print('n_true_positive:\t', n_correct)
    print('precision:\t[%.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f]' % (
        precision[0], precision[1], precision[2], precision[3], 
        precision[4], precision[5], precision[6]))
    print('recall:\t\t[%.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f]' % (
        recall[0], recall[1], recall[2], recall[3], 
        recall[4], recall[5], recall[6]))
    print('f1:\t\t[%.4f, %.4f, %.4f, %.4f, %.4f, %.4f, %.4f]' % (
        f1[0], f1[1], f1[2], f1[3], f1[4], f1[5], f1[6]))
    print('Micro F1: %.4f ' % (micro_f1))

In [ ]:
test("en_model_save/en_model_save_acc_06_260.857399910031489")

In [9]:
def make_sample(test_path, pretrained_model):
    result_file = './Friends/en_sample_0626.csv'
    fw = open(result_file, 'w')
    wr = csv.writer(fw)
    wr.writerow(["Id","Predicted"])
    
    ids = []
    sentences = []

    with open(test_path, 'r', encoding='CP949') as fr:
        rdr = csv.reader(fr)
        count_f = 0
        for line in rdr:
            if count_f == 0:
                count_f = 1
            else:
                ids.append(line[0])
                sentences.append(line[1])

    
    if not torch.cuda.is_available():
        raise NotImplementedError()
    hparams = type('', (object,), EMOTIONX_MODEL_HPARAMS)() # dict to class
    hparams.n_appear = [1, 1, 1, 1, 1, 1, 1, 1] # not to be used
    hparams.dropout = 0

    print('preprocessing...')
    tokenizer = BertTokenizer.from_pretrained(hparams.bert_type)
    inputs = []
    for dialog in sentences:
        tokenized_dialog = []
        tokenized_utter = tokenizer.tokenize(dialog.lower())
        tokenized_ids = tokenizer.convert_tokens_to_ids(tokenized_utter)
        if len(tokenized_dialog + tokenized_utter) + 1 > hparams.max_input_len:
            print('[CAUTION] over max_input_len: ', utter['utterance'])
            continue
        tokenized_dialog += tokenized_ids + [hparams.sep_id]
        inputs.append(tokenized_dialog)
  
    print('prediction...')
    model = EmotionX_Model(hparams)
    model.load_state_dict(torch.load(pretrained_model))
    model.cuda()
    model.eval()
    pred_list = []
    for i_test in range(len(inputs) // hparams.batch_size):
        batch = get_batch(inputs, hparams.batch_size, i_test)      
        logits = model(batch)[:, :-1] # trim the OOD column
        _, pred = torch.max(logits, dim=1)
        pred_list += pred.tolist()
    assert sum(inputs, []).count(102) == len(pred_list) # n_utter == n_pred

    print('labeling...')
    index_to_emotion = {0: 'neutral', 1: 'joy', 2: 'sadness',3: 'fear', 
                      4: 'anger', 5: 'surprise', 6: 'disgust', 7: 'non-neutral'}
    
    for i in range(len(pred_list)):
        pred_str = index_to_emotion[pred_list[i]]
        wr.writerow([ids[i], pred_str])

            
    fw.close()

In [10]:
make_sample('Friends/en_sample.csv', "en_model_save/en_model_save_acc_06_200.8547008547008547")

preprocessing...
prediction...
[1, 1, 1, 1, 1, 1, 1, 1]
labeling...
